### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

**Задание 1 (3 балла)**
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

**Задание 2 (3 балла)**
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

**Задание 3 (2 балла)**
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

**Задание 4 (2 балла)**
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self,
                 min_samples_split=2,
                 max_depth=None,
                 sufficient_share=1.0,
                 criterion='gini',
                 max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.impurity_measure = self.__gini_imp_measure
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.impurity_measure = self.__entropy_imp_measure
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.impurity_measure = self.__misclass_imp_measure
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    # обозначим прирост информации IG как IG(x) = I(S) - (phi(x))
    # логично, что влиять на S на текущем шаге мы уже не можем
    # т.е. имеем const - phi(x), т.е. чтобы максимизировать прирост информации
    # нужно минимизровать phi(x). В методах ниже считается как раз phi(x)
    # (взвешенная сумма I(S_l) и I(S_r))
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum(l_c ** 2 / l_s.reshape(-1, 1) +
                          r_c ** 2 / r_s.reshape(-1, 1),
                          axis=1) / (l_s + r_s)

    def __gini_imp_measure(self, p_c, p_s):
        p_c = p_c.astype(float)
        p_s = p_s.astype(float)
        return 1 - np.sum((p_c / p_s) ** 2)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum(l_c * np.log(l_c / l_s.reshape(-1, 1) +
                                    np.finfo(float).eps) +
                       r_c * np.log(r_c / r_s.reshape(-1, 1) +
                                    np.finfo(float).eps),
                       axis=1) / (l_s + r_s)

    def __entropy_imp_measure(self, p_c, p_s):
        p_c = p_c.astype(float)
        p_s = p_s.astype(float)
        return -np.sum((p_c / p_s) * np.log(p_c / p_s))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(l_c, axis=1) + np.max(r_c, axis=1)) / (l_s + r_s)

    def __misclass_imp_measure(self, p_c, p_s):
        p_c = p_c.astype(float)
        p_s = p_s.astype(float)
        return 1 - np.max(p_c / p_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.asarray(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:np.sqrt(n_feature).astype(int)]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.asarray(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:np.log2(n_feature).astype(int)]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # "...количественный признак сортируется по возрастанию..."
        x_sorted, y_sorted = self.__sort_samples(x, y)

        # "...проверяются только те пороги,
        # при которых целевой признак меняет значение..."
        borders = np.where(y_sorted[1:] != y_sorted[:-1])[0] + 1

        # шаблон говорит о распределении объектов классов по порогам
        # (нужен чтобы разом посчитать G_function)
        # сначала считаем количество объектов внутри бордеров
        classes_in_a_row = borders - np.append(np.array([0]), borders[:-1])

        template = np.zeros((borders.shape[0], self.num_class))

        # непосредственно указываем какому классу принадлежат объекты внутри
        template[np.arange(borders.shape[0]), y_sorted[borders - 1]] = 1

        # указываем количество объектов класса, находящихся между бордерами
        template *= classes_in_a_row.reshape(-1, 1)

        # "...для каждого порога строится дерево глубины 1..."
        l_c = np.cumsum(template, axis=0)
        r_c = np.bincount(y_sorted, minlength=self.num_class) - l_c
        l_s = np.sum(l_c, axis=1)
        r_s = y_sorted.shape[0] - l_s

        # "...считается насколько снизилась энтропия
        # (или неопределенность Джини)..."
        gs = self.G_function(l_c, l_s, r_c, r_s)

        # "...и выбираются только лучшие пороги,
        # c которыми стоит сравнивать количественный признак."
        idx = gs.argmin()
        before_border = x_sorted[int(l_s[idx])-1]
        after_border = x_sorted[int(l_s[idx])]
        best_threshold = np.mean([before_border, after_border])
        # © статья ODS по деревьям на хабре,
        # сильно помогла вспомнить что было на лекции
        return gs[idx], best_threshold

    def __fit_node(self, x, y, node_id, depth):
        if (depth == self.max_depth or
                x.shape[0] <= self.min_samples_split or
                np.unique(y).shape[0] == 1):
            self.tree[node_id] = (self.__class__.LEAF_TYPE,
                                  np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.shape[0])
            return

        features_ids = self.get_feature_ids(x.shape[1])

        # чтобы не было проблем с шейпами и можно было
        # по argmin вытащить нужный id фичи
        feature_threshold_pairs = np.zeros((2, x.shape[1])) + np.inf

        feature_threshold_pairs[:, features_ids] = \
            np.apply_along_axis(self.__find_threshold,
                                0,
                                x[:, features_ids],
                                y)

        # нашли как будем делить
        best_split_feature_id = \
            np.argmin(feature_threshold_pairs[0, :])
        best_split_feature_threshold = \
            feature_threshold_pairs[1, best_split_feature_id]
        # поделили
        x_l, x_r, y_l, y_r = self.__div_samples(x,
                                                y,
                                                best_split_feature_id,
                                                best_split_feature_threshold)

        # по пути считаем feature importances
        self.feature_importances_[best_split_feature_id] += \
            self.impurity_measure(np.bincount(y),
                                  np.asarray(y.shape[0])) - \
            (float(y_l.shape[0]) *
                self.impurity_measure(np.bincount(y_l),
                                      np.asarray(y_l.shape[0])) +
             float(y_r.shape[0]) *
                self.impurity_measure(np.bincount(y_r),
                                      np.asarray(y_r.shape[0]))) / \
            float(y.shape[0])

        # вдруг получилось так, что можно положить все в листовую ноду
        if x_l.shape[0] == 0 or x_r.shape[0] == 0:
            self.tree[node_id] = (self.__class__.LEAF_TYPE,
                                  np.bincount(y).argmax(),
                                  np.bincount(y).astype(float) / y.shape[0])
            return

        # записали то как поделили
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE,
                              best_split_feature_id,
                              best_split_feature_threshold)

        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)

        return

    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1])
        self.num_class = np.unique(y).shape[0]
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.shape[0]

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 1.8 ms, sys: 178 µs, total: 1.97 ms
Wall time: 1.18 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 31.1 ms, sys: 0 ns, total: 31.1 ms
Wall time: 30 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9487179487179486

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9487179487179486

## Подготовка данных Speed Dating Data 

In [9]:
def one_hot(data, feature):
    for code in np.sort(data[feature].unique()):
        data[feature + '=' + str(code)] = \
            (data[feature] == code).astype(np.float)
    data = data.drop([feature], axis=1)

In [10]:
df = pd.read_csv('Speed Dating Data.csv', sep=',', encoding='latin1')
print('df.shape =', df.shape)

df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position',
              'positin1', 'order', 'partner', 'age_o',
              'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o',
              'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o',
              'shar_o', 'like_o', 'prob_o', 'met_o'], axis=1)

df = df.dropna(subset=['age'])

df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = df.drop(['field'], axis=1)
one_hot(df, 'field_cd')
df = df.drop(['field_cd'], axis=1)

df = df.drop(['undergra'], axis=1)

df.loc[:, 'mn_sat'] = \
    df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)

df.loc[:, 'tuition'] = \
    df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)

one_hot(df, 'race')

df = df.dropna(subset=['imprelig', 'imprace'])
df = df.drop(['from', 'zipcode'], axis=1)

df.loc[:, 'income'] = \
    df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)

df = df.dropna(subset=['date'])

one_hot(df, 'goal')

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = df.drop(['career'], axis=1)
one_hot(df, 'career_c')

df = df.drop(['sports', 'tvsports', 'exercise', 'dining',
              'museums', 'art', 'hiking', 'gaming', 'clubbing',
              'reading', 'tv', 'theater', 'movies', 'concerts',
              'music', 'shopping', 'yoga'], axis=1)

df = df.drop(['expnum'], axis=1)

df.loc[:, 'temp_totalsum'] = \
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
               'amb1_1', 'shar1_1']].sum(axis=1)

df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
           'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                'fun1_1', 'amb1_1', 'shar1_1']].T /
     df.loc[:, 'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = \
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
               'amb2_1', 'shar2_1']].sum(axis=1)

df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
           'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                'fun2_1', 'amb2_1', 'shar2_1']].T /
     df.loc[:, 'temp_totalsum'].T).T * 100

df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)

df = df.drop(['wave'], axis=1)

df_male = \
    df.query('gender == 1')\
    .drop_duplicates(subset=['iid', 'pid'])\
    .drop(['gender'], axis=1)\
    .dropna()

df_female = \
    df.query('gender == 0')\
    .drop_duplicates(subset=['iid'])\
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
    .dropna()

df_female.columns = df_female.columns + '_f'

df_female = df_female.drop(['pid_f'], axis=1)

df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
print('df_pair.shape =', df_pair.shape)

df.shape = (8378, 195)
df_pair.shape = (3999, 157)


In [11]:
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [13]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [14]:
%time clf.fit(X_train, y_train)

CPU times: user 102 ms, sys: 0 ns, total: 102 ms
Wall time: 102 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [15]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.34 s, sys: 0 ns, total: 1.34 s
Wall time: 1.34 s


## Проверка качества работы на Speed Dating Data

In [16]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5747696669029057

In [17]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.551264367816092

## Задание 3

## Задание 4

In [18]:
print('DecisionTreeClassifier')
print(list(df_pair.columns[1:][np.flip(np.argsort(clf.feature_importances_))])[:10])
print()
print('MyDecisionTreeClassifier')
print(list(df_pair.columns[1:][np.flip(np.argsort(my_clf.feature_importances_))])[:10])

DecisionTreeClassifier
['int_corr', 'age', 'income_f', 'sinc1_1_f', 'exphappy', 'attr3_1', 'fun2_1_f', 'attr1_1', 'intel2_1', 'shar2_1_f']

MyDecisionTreeClassifier
['int_corr', 'income', 'shar2_1_f', 'intel2_1_f', 'intel1_1', 'tuition_f', 'fun2_1_f', 'amb3_1_f', 'date_f', 'exphappy_f']


## Задание 5

In [19]:
forest = RandomForestClassifier()

params = {
    'max_depth': [6, 12, 18, 24, 36, 48],
    'min_samples_split': [2, 4, 6, 8, 10, 20, 40],
    'n_estimators': [1, 5, 10, 15, 20, 40]
}

searcher = GridSearchCV(forest, params, cv=3)
searcher.fit(X_train, y_train)

print(searcher.best_params_)


{'max_depth': 36, 'min_samples_split': 8, 'n_estimators': 40}


In [20]:
forest = RandomForestClassifier(max_depth=36,
                                min_samples_split=8,
                                n_estimators=40)
forest.fit(X_train, y_train)
print(f1_score(y_pred=forest.predict(X_test),
               y_true=y_test,
               average='macro'))

0.5772994129158513
